In [1]:
import pandas as pd
from pandasgui import show
import os

notebook_path = os.getcwd()
print(notebook_path)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
flow_pt_to_es = pd.read_parquet("../.data_cache/flow_pt_to_es.parquet")
flow_es_to_pt = pd.read_parquet("../.data_cache/flow_es_to_pt.parquet")
generation_pt = pd.read_parquet("../.data_cache/generation_pt.parquet")
generation_es = pd.read_parquet("../.data_cache/generation_es.parquet")
show(flow_pt_to_es)
show(flow_es_to_pt)
show(generation_pt)
show(generation_es)


PandasGUI INFO — pandasgui.gui — Opening PandasGUI


c:\Users\diogo\OneDrive\Ambiente de Trabalho\test\electricity_consumption_mix\tools


PandasGUI INFO — pandasgui.gui — Opening PandasGUI
PandasGUI INFO — pandasgui.gui — Opening PandasGUI
PandasGUI INFO — pandasgui.gui — Opening PandasGUI


In [9]:

df1 = pd.DataFrame({'A': [1, 2, 3]})
df2 = pd.DataFrame({'B': [1, 2, 3]})
df3 = pd.DataFrame({'C': [1, 2, 3]})

df4 = df1.add(df2, fill_value=0)
df4

,A,B
0,1.0,1.0
1,2.0,2.0
2,3.0,3.0


In [11]:
df3["C"]*df4

,A,B,0,1,2
0,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN


In [12]:
df4.multiply(df3["C"], axis=0)

,A,B
0,1.0,1.0
1,4.0,4.0
2,9.0,9.0


In [14]:
import numpy as np
df3["C"].to_numpy()*df4

ValueError: Unable to coerce to Series, length must be 2: given 3